# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [4]:
import pandas, numpy
from scipy.spatial.distance import pdist, squareform

In [10]:
data = pandas.read_csv('../data/customer_product_sales.csv')

In [11]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
customer = data.groupby(['CustomerID','ProductName']).sum()
customer

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [28]:
#Ejemplo de la documentación
#table = pd.pivot_table(df, values='D', index=['A', 'B'],
#columns=['C'], aggfunc=np.sum, fill_value=0)

matrix = pandas.pivot_table(data, index="ProductName", columns="CustomerID", values="Quantity", aggfunc=sum,fill_value=0)
#matrix.fillna(0, inplace=True)

matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [30]:
simi_matrix = pandas.DataFrame(1/(1 + squareform(pdist(matrix.T, 'euclidean'))), index=matrix.columns, columns=matrix.columns)
simi_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [42]:
topcustomers = pandas.DataFrame(simi_matrix.loc[200].sort_values(ascending=False)).head(5).reset_index()
topcustomers

,CustomerID,200
0,200,1.000000
1,3317,0.084959
2,1072,0.083972
3,3535,0.082403
4,1920,0.082100


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [49]:
similar = data[data['CustomerID'].isin(topcustomers['CustomerID'].unique())]
similar.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
495,200,Rene,Mercado,4820933,432,Beer - Original Organic Lager,1
531,1920,Cheri,Mason,1733830,432,Beer - Original Organic Lager,1
663,200,Rene,Mercado,1924085,428,Wine - Ruffino Chianti,1
1042,3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [63]:
custumers_purchase= pandas.DataFrame(similar.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)).reset_index()
custumers_purchase

,ProductName,Quantity
0,Lamb - Ground,6
1,"Pepper - Black, Whole",4
2,Crush - Cream Soda,3
3,Tahini Paste,3
4,Pork - Kidney,3
...,...,...
207,"Pepper - Paprika, Hungarian",1
208,Pate - Cognac,1
209,Pastry - Choclate Baked,1
210,"Pasta - Penne, Rigate, Dry",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [75]:
top_purchased = custumers_purchase [:5]
top_purchased

,ProductName,Quantity
0,Lamb - Ground,6
1,"Pepper - Black, Whole",4
2,Crush - Cream Soda,3
3,Tahini Paste,3
4,Pork - Kidney,3


In [71]:
not_purchased = custumers_purchase[custumers_purchase.Quantity<=1]
not_purchased.head(5)

,ProductName,Quantity
60,Curry Paste - Madras,1
61,Juice - Apple Cider,1
62,Ketchup - Tomato,1
63,Kellogs Special K Cereal,1
64,Cup - Translucent 7 Oz Clear,1


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [82]:
recomendation = {}
data.CustomerID.unique()

array([61288, 77352, 40094, 23548, 78981, 83106, 11253, 35107, 15088,
       26031, 86595, 46715, 91013, 69931, 19753,  9546, 10224, 72846,
       28976,  3267, 19260, 76530, 30196, 48818, 15166, 11443, 79060,
       82597, 62411, 42611, 45420, 83495, 47122, 30815, 53198, 18591,
       62034,  1428,  2187, 55162, 22345, 81876, 48641, 81034, 29681,
       87947, 28867, 69967, 94420, 83158, 84364, 44742,  7772, 47303,
       75791, 77756, 97324, 13912, 52700, 21734,  6817, 21095, 28875,
       89588,  1034, 54827, 29287, 11102, 52155, 50748, 88552, 80938,
       60481, 92168, 66232, 64348, 37510, 44408, 89254,   477, 51483,
       97201, 42157, 52091, 86065, 18027, 93882, 42303, 32010, 53997,
       63033, 76475, 98185, 84779, 82146, 54495,  3903, 79854, 25449,
       85496, 95017, 33759, 43652, 35873, 97769,   891, 85161, 25327,
       25624, 68416, 44090, 71595, 68336, 38497, 10016, 36751, 58421,
       45498, 50774, 79997, 45313, 55715, 40901,  1577,  3909, 18480,
       41405, 85552,

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
##Por prisa se queda pendiente

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
##Por prisa se queda pendiente